In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Specify career len to export file for
# CAREER_LENGTH = 15
# Specify how long is the early career. Impacts which papers we take into account for early productivity and quality
# EARLY_CAREER_LEN = 3
EARLY_CAREER_LEN_LIST = [1, 2, 3, 4, 5]
EARLY_CAREER_LEN_LIST = [3,5,7,9,11,12]
# For early career work, when do we stop counting citations. Impacts recognition
# RECOGNITION_CUT_OFF = 5
RECOGNITION_CUT_OFF_LIST = [3, 4, 5, 6, 7, 8, 9]
RECOGNITION_CUT_OFF_LIST = [3, 5]
RECOGNITION_CUT_OFF_LIST = [3,5,7,9,11,12]
# Success after 15 years. Impacts when we stop counting citations
SUCCESS_CUTOFF = 15
# Length of observed career for dropouts
# (1-3), middle career (4-9), late career (10-15)

# TODO: for multiple dropout intervals code does not work!!!
CAREER_LENGTH_DROPOUTS_LIST = [ (0, 15), (0, 3), (4, 9), (10, 15)] #,
CAREER_LENGTH_DROPOUTS_LIST = [(0, 15)]
# CAREER_LENGTH_DROPOUTS = 15
INACTIVE_TIME_DROPOUTS = 10

# Specify the first and last year we consider in our analysis
# TODO: Should we specify last start year? 
START_YEAR = 1970
# LAST_START_YEAR = 2018 - max(CAREER_LENGTH, CAREER_LENGTH_DROPOUTS)

In [4]:
# assert(INACTIVE_TIME_DROPOUTS < CAREER_LENGTH_DROPOUTS), "Time observed for dropouts has to be smaller than the whole window!"

In [5]:
# assert(CAREER_LENGTH >= EARLY_CAREER_LEN), "Early career len too long"

## 1. Load data

In [6]:
authorPublicationData = pd.read_csv('./data/author_publications_2017_asiansAsNone.txt')

authorPublicationData.head()

,author,year,pub_id
0,graeme earl,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821
1,gareth beale,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821
2,m. nasser,2009,001c8744-73c4-4b04-9364-22d31a10dbf1
3,faisal zaman,2009,001c8744-73c4-4b04-9364-22d31a10dbf1
4,altaf hossain,2009,001c8744-73c4-4b04-9364-22d31a10dbf1


In [7]:
print(authorPublicationData.shape)
# same as dropping author, pub_id and year
authorPublicationData.drop_duplicates(subset=['author','pub_id'], inplace=True)
print(authorPublicationData.shape)

(9471668, 3)
(9467149, 3)


In [8]:
authorCitationsData = pd.read_csv('./data/citations_2017_asiansAsNone.txt')
authorCitationsData.head()

,id1,id2,year
0,1fcd3d7f-1bb7-4347-914e-e23adb1e1a59,5065f1e8-0e2e-4e20-9866-2b5067d321dc,1990
1,1fcd3d7f-1bb7-4347-914e-e23adb1e1a59,cf862fd8-9204-4e44-9639-2d067c247539,1990
2,1fcd3d7f-1bb7-4347-914e-e23adb1e1a59,f19127f8-3d3a-4a9e-9580-c1d3e1e05141,1990
3,03b1d66a-b1fc-4d99-9904-9405695cc2ae,09991de0-c00f-49cf-a88a-6515943b0843,1994
4,03b1d66a-b1fc-4d99-9904-9405695cc2ae,156173f6-35eb-44f6-bc75-7c205153b6f0,1994


In [9]:
print(authorCitationsData.shape)
authorCitationsData.drop_duplicates(inplace=True)
print(authorCitationsData.shape)

(25161834, 3)
(25161834, 3)


In [10]:
print('Authors#      - ',authorPublicationData['author'].nunique())
print('Years#        - ',authorPublicationData['year'].nunique())
print('Publications# - ',authorPublicationData['pub_id'].nunique())

Authors#      -  1704919
Years#        -  83
Publications# -  3078230


## 2. Career length and dropouts

In [11]:
groupByAuthor = authorPublicationData.groupby(['author'])

groupByAuthorMinYearData = groupByAuthor['year'].min()
groupByAuthorMaxYearData = groupByAuthor['year'].max()
groupByAuthorCountPublicationsData = groupByAuthor['pub_id'].count()

In [12]:
authorGroupedData = groupByAuthorMinYearData.to_frame(name='start_year')
authorGroupedData['end_year'] = groupByAuthorMaxYearData
authorGroupedData['total_num_pub'] = groupByAuthorCountPublicationsData
authorGroupedData = authorGroupedData.reset_index()
print('Total rows -                ', authorGroupedData.shape)

authorGroupedData = authorGroupedData[authorGroupedData["start_year"] >= START_YEAR]
print('After removing all < 1970 - ', authorGroupedData.shape)

authorGroupedData = authorGroupedData.drop_duplicates()
print('After removing duplicates - ', authorGroupedData.shape)

authorGroupedData = authorGroupedData.dropna(how='any')
print("After droping na -          ", authorGroupedData.shape)

authorGroupedData.head()

Total rows -                 (1704919, 4)
After removing all < 1970 -  (1699352, 4)
After removing duplicates -  (1699352, 4)
After droping na -           (1699352, 4)


,author,start_year,end_year,total_num_pub
0,'maseka lesaoana,2001,2015,2
1,(max) zong-ming cheng,2009,2011,2
2,(zhou) bryan bai,2011,2012,2
3,a aart blokhuis,1992,2005,2
4,a ahrabian,2017,2017,1


In [13]:
# Adding 1 here to have career length be at least one. So 3 years career means year1, year2, year3.
authorGroupedData["career_length"] = authorGroupedData['end_year'] - authorGroupedData['start_year'] + 1

In [14]:
credible_authors = authorGroupedData

### Label authors that drop out

In [15]:
print(f"Label authors with {INACTIVE_TIME_DROPOUTS} years inacitivity in a {CAREER_LENGTH_DROPOUTS_LIST} years window as dropouts")

Label authors with 10 years inacitivity in a [(0, 15)] years window as dropouts


In [16]:
combined_pubs = authorPublicationData.merge(credible_authors[['author', 'start_year']], on='author', how='inner')

print(combined_pubs.head())
print(combined_pubs.shape)

        author  year                                pub_id  start_year
0  graeme earl  2011  001c58d3-26ad-46b3-ab3a-c1e557d16821        2009
1  graeme earl  2011  6af505cf-727f-47f9-afea-772af2928086        2009
2  graeme earl  2009  3ac1f00f-6830-42e9-b6f9-944f00a71b3c        2009
3  graeme earl  2009  3133714c-f979-4d84-9224-97361cf053ab        2009
4  graeme earl  2014  71d3749b-3e35-461b-86c8-920c42d5ebe8        2009
(9413377, 4)


In [17]:
def list_append(lst, item):
    lst.append(item)
    return lst

In [18]:
%%time
for start, end in CAREER_LENGTH_DROPOUTS_LIST:
    combined_pubs_grouped = combined_pubs[(combined_pubs.year >= combined_pubs.start_year + start) &
                                    (combined_pubs.year <= combined_pubs.start_year + end)]   
    # for every 2 consecutive years the author has published (nxt and prev) find a difference (absence time)
    # we artificially add two value: career start + 15 and career start, as limiters of our observation window
    # this will add 0 values in the begining for the first year
    combined_pubs_grouped = combined_pubs_grouped.groupby('author')['year', 'start_year'].apply(
        lambda x: [nxt - prev for prev, nxt in zip(sorted(list_append(list(x['year']),x['start_year'].iloc[0]+start)), 
                                                   sorted(list_append(list(x['year']),x['start_year'].iloc[0]+  end)))])
    combined_pubs_grouped = combined_pubs_grouped.reset_index()
    combined_pubs_grouped.rename({0:f'absence_list-{start}-{end}'}, inplace=True, axis='columns')
    combined_pubs_grouped[f'max_absence-{start}-{end}'] = combined_pubs_grouped[f'absence_list-{start}-{end}'].apply(max)
    combined_pubs_grouped[f'avg_absence-{start}-{end}'] = combined_pubs_grouped[f'absence_list-{start}-{end}'].apply(np.mean)
    
    credible_authors = credible_authors.merge(combined_pubs_grouped[['author', f'max_absence-{start}-{end}', 
                                                                     f'avg_absence-{start}-{end}']], on='author', how='left')
    credible_authors[f'max_absence-{start}-{end}'] = credible_authors[f'max_absence-{start}-{end}'].fillna(end-start+1)
    credible_authors[f'avg_absence-{start}-{end}'] = credible_authors[f'avg_absence-{start}-{end}'].fillna(end-start+1)
    
    # TODO: Should i also add the start year into the calculation? Now i only have end year included

CPU times: user 13min 15s, sys: 6.56 s, total: 13min 22s
Wall time: 14min 4s


In [19]:
# display(combined_pubs_grouped.head())
# display(combined_pubs_grouped.shape)

In [20]:
credible_authors['dropped_after_10'] = credible_authors['max_absence-0-15'].apply(lambda x: False if x < 10 else True)

In [21]:
credible_authors['max_absence-0-15'].value_counts(dropna=False)

15    915304
14    118959
13    103782
12     84939
11     74752
10     66023
9      61777
8      60327
7      47302
6      42319
5      36687
4      32426
3      27540
2      21247
1       5968
Name: max_absence-0-15, dtype: int64

In [22]:
credible_authors.shape

(1699352, 8)

In [23]:
credible_authors['dropped_after_10'].value_counts()

True     1363759
False     335593
Name: dropped_after_10, dtype: int64

In [24]:
credible_authors.columns

Index(['author', 'start_year', 'end_year', 'total_num_pub', 'career_length',
       'max_absence-0-15', 'avg_absence-0-15', 'dropped_after_10'],
      dtype='object')

### Gender

In [25]:
gender = pd.read_csv('./data/name_gender_2017_asiansAsNone_nodup.txt')
credible_authors = credible_authors.merge(gender, left_on='author', right_on='name', how='left')
credible_authors.drop('name', axis=1, inplace=True)

credible_authors.gender.value_counts()

m       793766
none    650086
f       255500
Name: gender, dtype: int64

### Save filtered data about authors, and cleaned publications

In [26]:
credible_authors.to_csv('derived-data/authors-scientific.csv', index=False, encoding='utf-8')
credible_authors.head()

,author,start_year,end_year,total_num_pub,career_length,max_absence-0-15,avg_absence-0-15,dropped_after_10,gender
0,'maseka lesaoana,2001,2015,2,15,14,5.0,True,none
1,(max) zong-ming cheng,2009,2011,2,3,13,5.0,True,none
2,(zhou) bryan bai,2011,2012,2,2,14,5.0,True,none
3,a aart blokhuis,1992,2005,2,14,13,5.0,True,none
4,a ahrabian,2017,2017,1,1,15,7.5,True,none


In [27]:
authorPublicationData.to_csv('derived-data/author-publications.csv', index=False)

## 3. Generate a new citation network

### Generate Author->Paper network

In [28]:
# We need data about how many times an author has been cited
# For every authors publication, i merge all citations
final_citation_count_from_ids = authorPublicationData.merge(authorCitationsData, left_on='pub_id', 
                                                            right_on='id2', how='inner', suffixes=('_pub', '_cit'))

In [29]:
print(final_citation_count_from_ids.shape)

(75912535, 6)


In [30]:
final_citation_count_from_ids.drop_duplicates(inplace=True)

In [31]:
print(final_citation_count_from_ids.shape)

(75912535, 6)


#### Remove errors in citation data (years published vs years cited)

In [32]:
# Published before cited - NORMAL
print(final_citation_count_from_ids.shape)
num_normal = final_citation_count_from_ids[final_citation_count_from_ids.year_pub <= final_citation_count_from_ids.year_cit].shape
print(num_normal)

(75912535, 6)
(75392625, 6)


In [33]:
# Published after cited - WRONG
num_wrong = final_citation_count_from_ids[final_citation_count_from_ids.year_pub > final_citation_count_from_ids.year_cit].shape
print(num_wrong)

(519910, 6)


In [34]:
print("Percentage of citations to be removed: ", num_wrong[0]*100/(num_normal[0]+num_wrong[0]))
print("Less than one percent")

Percentage of citations to be removed:  0.6848803033649186
Less than one percent


In [35]:
cit_wrong_df = final_citation_count_from_ids[final_citation_count_from_ids.year_pub > final_citation_count_from_ids.year_cit]

In [36]:
cit_wrong = final_citation_count_from_ids[final_citation_count_from_ids.year_pub > final_citation_count_from_ids.year_cit].index

In [37]:
final_citation_count_from_ids.drop(cit_wrong, inplace=True)

assert num_normal[0] == final_citation_count_from_ids.shape[0], "The number of citations doesnt match"

In [38]:
final_citation_count_from_ids.columns

Index(['author', 'year_pub', 'pub_id', 'id1', 'id2', 'year_cit'], dtype='object')

#### Save

In [39]:
# final_citation_count_from_ids.to_csv('./data/authors_cited_by_papers_2017_asiansAsNone_by_daniel.txt',
#                                      columns=['author', 'year_pub', 'pub_id', 'id1', 'year_cit'], index=False)

final_citation_count_from_ids[['author', 'id1', 'id2', 'year_cit']].drop_duplicates().to_csv('derived-data/author-paper-citations-cleaned.csv', 
                                                                                   index=False)

# final_citation_count_from_ids.drop_duplicates(subset=['author_cited', 'pub_id_cited', 'pub_id_citing', 'author_citing'],
#                                               inplace=True)

In [41]:
# drop duplicates on id1,id2 because we only care about paper->paper citations
paper_citation_count = final_citation_count_from_ids.drop_duplicates(subset=['id1', 'id2']).groupby('id2')['id1'].count()
paper_citation_count.to_csv('derived-data/paper-citation-count.csv')

Its important to keep using this file for citations. As it has bad entries removed

### Generate Author->Author network

In [42]:
# # To generate an author->author citation network
# # For every citation an author received, merge in all the authors that cited him
# # TODO Should we be filtering citations? Do we include them all? We do drop ones cited before published
# final_citation_count_from_ids = final_citation_count_from_ids.merge(authorPublicationData, left_on='id1', 
#                                                                     right_on='pub_id', how='left', suffixes=('_cited', '_citing'))
# final_citation_count_from_ids.to_csv('./derived-data/author-author-citations.csv',
#                                      columns=['author_cited', 'author_citing', 'year_cit'], index=False)

In [43]:
final_citation_count_from_ids.head()

,author,year_pub,pub_id,id1,id2,year_cit
0,graeme earl,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821,16870052-7e2e-4bbc-beeb-21e493bea221,001c58d3-26ad-46b3-ab3a-c1e557d16821,2015
1,gareth beale,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821,16870052-7e2e-4bbc-beeb-21e493bea221,001c58d3-26ad-46b3-ab3a-c1e557d16821,2015
2,m. nasser,2009,001c8744-73c4-4b04-9364-22d31a10dbf1,49ec0c15-7a9d-4378-a26a-f5063567b519,001c8744-73c4-4b04-9364-22d31a10dbf1,2013
3,m. nasser,2009,001c8744-73c4-4b04-9364-22d31a10dbf1,ae0b4e3d-305d-4fb8-9847-60dcc9a9ec44,001c8744-73c4-4b04-9364-22d31a10dbf1,2014
4,faisal zaman,2009,001c8744-73c4-4b04-9364-22d31a10dbf1,49ec0c15-7a9d-4378-a26a-f5063567b519,001c8744-73c4-4b04-9364-22d31a10dbf1,2013


### Group citations over authors and years

In [44]:
citations_year_auth = final_citation_count_from_ids.groupby(['author', 'year_cit'])['id1'].count()


In [45]:
citations_year_auth.head()

author            year_cit
'maseka lesaoana  2002        1
                  2003        1
                  2004        1
                  2005        2
                  2006        4
Name: id1, dtype: int64

In [46]:
citations_year_auth = citations_year_auth.reset_index()
citations_year_auth = citations_year_auth.rename(columns={'id1':'cit_count'})

# citations_year_auth[['author', 'year_cit', 'cit_count']].to_csv('derived-data/authors-perYear-citations-atleast-'+str(CAREER_LENGTH)+'-year.csv', index=False)
citations_year_auth[['author', 'year_cit', 'cit_count']].to_csv('derived-data/authors-perYear-citations.csv', index=False)

In [47]:
citations_year_auth = citations_year_auth.groupby(['author', 'year_cit'])['cit_count'].sum()

## Early career analysis

In [49]:
combined = final_citation_count_from_ids.merge(credible_authors[['author', 'start_year']], on='author', how='inner')

In [50]:
combined.shape

(74452472, 7)

In [51]:
combined.head()

,author,year_pub,pub_id,id1,id2,year_cit,start_year
0,graeme earl,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821,16870052-7e2e-4bbc-beeb-21e493bea221,001c58d3-26ad-46b3-ab3a-c1e557d16821,2015,2009
1,graeme earl,2011,6af505cf-727f-47f9-afea-772af2928086,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009
2,graeme earl,2009,3ac1f00f-6830-42e9-b6f9-944f00a71b3c,523e22e9-0b2f-439f-bad6-dd8037364f44,3ac1f00f-6830-42e9-b6f9-944f00a71b3c,2010,2009
3,graeme earl,2009,3133714c-f979-4d84-9224-97361cf053ab,001c58d3-26ad-46b3-ab3a-c1e557d16821,3133714c-f979-4d84-9224-97361cf053ab,2011,2009
4,graeme earl,2009,3133714c-f979-4d84-9224-97361cf053ab,523e22e9-0b2f-439f-bad6-dd8037364f44,3133714c-f979-4d84-9224-97361cf053ab,2010,2009


In [52]:
combined.drop_duplicates(subset=['author', 'id1', 'id2'], inplace=True)

### Early degree

In [53]:
for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
    combined_early_degree = combined[(combined.year_pub < combined.start_year + EARLY_CAREER)]

    combined_early_degree = combined_early_degree.drop_duplicates(subset=['author', 'pub_id'])

    combined_early_degree = combined_early_degree[['author', 'pub_id']]

    # authors_per_paper = combined_early_degree.groupby('pub_id')['author'].count().reset_index()
    # authors_per_paper.rename({"author":"early_career_degree"}, axis='columns', inplace=True)

    combined_early_degree = combined_early_degree.merge(combined, on='pub_id')

    combined_early_degree = combined_early_degree[combined_early_degree.author_x != combined_early_degree.author_y]
    combined_early_degree = combined_early_degree.drop_duplicates(subset=['author_x', 'author_y'])

    combined_early_degree = combined_early_degree.groupby('author_x')['author_y'].count().reset_index()

    combined_early_degree.rename({"author_x":"author", "author_y": f"early_career_degree_{EARLY_CAREER}"}, 
                                 axis='columns', inplace=True)

    credible_authors = credible_authors.merge(combined_early_degree, on='author', how='left')
    credible_authors[f"early_career_degree_{EARLY_CAREER}"] = credible_authors[f"early_career_degree_{EARLY_CAREER}"].fillna(0)

In [54]:
combined_early_degree.sample(10)

,author,early_career_degree_12
1025947,walter w. immerzeel,6
164036,chris m. d'eramo,2
690602,miroslaw rojewski,3
786033,ping-fan dai,4
699214,mona riabacke,3
1068914,yasuyuki yamada,4
48880,amit m. joshi,1
483671,jorge munoz-gama,11
23250,akihito fujita,2
991629,tony benavides,9


### Early quality

In [55]:
for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
    combined_early_quality = combined[(combined.year_pub < combined.start_year + EARLY_CAREER) &
             (combined.year_cit < combined.start_year + SUCCESS_CUTOFF)]

    early_career_quality = combined_early_quality.groupby('author')['id1'].count()

    early_career_quality = early_career_quality.rename(f'early_career_qual_{EARLY_CAREER}')
    early_career_quality = early_career_quality.reset_index()
    credible_authors = credible_authors.merge(early_career_quality, on='author', how='left')
    credible_authors[f'early_career_qual_{EARLY_CAREER}'] = credible_authors[f'early_career_qual_{EARLY_CAREER}'].fillna(0)

### Early recognition

In [56]:
for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
    for RECOGNITION_CUT in RECOGNITION_CUT_OFF_LIST:
        if RECOGNITION_CUT != EARLY_CAREER: continue
        early_career_recognition = combined[(combined.year_pub < combined.start_year + EARLY_CAREER) &
                 (combined.year_cit < combined.start_year + RECOGNITION_CUT)]
        early_career_recognition = early_career_recognition.groupby('author')['id1'].count()
        col_name = f'early_career_recognition_EC{EARLY_CAREER}_RC{RECOGNITION_CUT}'
        early_career_recognition = early_career_recognition.rename(col_name)
        early_career_recognition = early_career_recognition.reset_index()
        credible_authors = credible_authors.merge(early_career_recognition, on='author', how='left')
        credible_authors[col_name] = credible_authors[col_name].fillna(0)

### Final success

In [57]:
combined_succ_after_15y = combined[combined.year_cit < combined.start_year + SUCCESS_CUTOFF]

In [58]:
succ_after_15y = combined_succ_after_15y.groupby('author')['id1'].count()

succ_after_15y = succ_after_15y.rename('succ_after_15y')
succ_after_15y = succ_after_15y.reset_index()
credible_authors = credible_authors.merge(succ_after_15y, on='author', how='left')
credible_authors['succ_after_15y'] = credible_authors['succ_after_15y'].fillna(0)

### H index

In [ ]:
def h_index(citations):
    if len(citations) == 0: return 0
    if len(citations) == 1: return 1
    citations = sorted(citations, reverse=True)
    h_ind = 0
    for i, elem in enumerate(citations):
        if i+1 > elem:
            return i
        h_ind = i+1
    return h_ind

In [ ]:
for param in [*EARLY_CAREER_LEN_LIST, SUCCESS_CUTOFF]:

    combined_h_index = combined[combined.year_cit < combined.start_year + param]

    combined_h_index = combined_h_index.groupby(['author', 'pub_id'])['id1'].count()

    combined_h_index = combined_h_index.reset_index()

    combined_h_index = combined_h_index.groupby('author')['id1'].apply(lambda x: h_index(x.values))

    combined_h_index = combined_h_index.rename(f'h-index_{param}')

    credible_authors = credible_authors.merge(combined_h_index.reset_index(), on='author', how='left')
    credible_authors[f'h-index_{param}'] = credible_authors[f'h-index_{param}'].fillna(0)

In [100]:
# TODO: test h-index

In [ ]:
def get_early_career_prod(row, early_career_len):
    early_prod = 0
    author = row.author
    start_year = row['start_year']
    year_num_pub = author_year_numPub[author]
    for year in range(early_career_len):
        if start_year + year in year_num_pub:
            early_prod += year_num_pub[start_year + year]
    return early_prod

In [ ]:
author_year_numPub = authorPublicationData.groupby(['author', 'year'])['pub_id'].count()

In [ ]:
%%time
for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
    credible_authors[f'early_career_prod_{EARLY_CAREER}'] = credible_authors.apply(
        lambda x: get_early_career_prod(x, EARLY_CAREER), axis=1)

In [ ]:
def get_early_publications(row, early_career_len):
    # TODO remove
    early_pubs = []
    author = row.author
    start_year = row['start_year']
    year_pub_list = author_year_publications[author]
    for year in range(early_career_len):
        if start_year + year in year_pub_list:
            early_pubs.extend(year_pub_list[start_year + year])
    return early_pubs

In [ ]:
# author_year_publications = authorPublicationData.groupby(['author', 'year'])['pub_id'].apply(list)

In [ ]:
# %%time
# credible_authors['early_career_pubs'] = credible_authors.apply(lambda x: get_early_publications(x, EARLY_CAREER_LEN), axis=1)

In [ ]:
# final_citation_count = citations_year_auth.groupby('author')['cit_count'].sum()

# final_citation_count = final_citation_count.reset_index()

# credible_authors = credible_authors.merge(final_citation_count, on='author', how='left') 


# credible_authors.rename({'cit_count': 'final_citation_count'}, axis=1, inplace=True)

# credible_authors['final_citation_count'] = credible_authors['final_citation_count'].fillna(0)

### Early Coauthor max h-index

In [83]:
# for each paper in EC, calculate the h-index of all its authors
# This requires extra work
# We want to calculate the h index of coauthors at the time of publishing the paper
# for this we need an extra lookup table, where we store 
# all papers - authors - h-index at the time
# 

# final_citation_count_from_ids - we merge pub data with cit data, but "inner"
# this means we will not find papers with 0 citations in this df
# these papers dont impact the h-index, so this is okay

In [251]:
def author_h_index_in_year_X(authors, year_x):
#     print(year_x)
    combined_h = combined[(combined.year_cit < year_x) & (combined.author.isin(authors))]
    combined_h = combined_h.groupby(['author', 'pub_id']).agg({'id1': 'count'}).reset_index()
    author_hind_at_year = combined_h.groupby('author').agg({'id1': h_index}).reset_index()
    author_hind_at_year['year_pub'] = year_x
    author_hind_at_year = author_hind_at_year.rename({'id1': 'h-index'}, axis='columns')
    return author_hind_at_year

In [250]:
def author_h_index(author, year_x):
    combined_h = combined[(combined.year_cit < year_x) & (combined.author == author)]
    citations_count_list = combined_h.groupby(['pub_id']).agg({'id1': 'count'})['id1'].values
    return h_index(citations_count_list)

In [262]:
papers_authors.shape

(3308476, 2)

In [261]:
%%time
papers_authors = combined[['author', 'year_pub']].drop_duplicates(
    subset=['author', 'year_pub'])

CPU times: user 11.8 s, sys: 4.58 s, total: 16.4 s
Wall time: 11.4 s


In [263]:
%%time
all_authors_hind = pd.DataFrame(columns=['author', 'h-index', 'year_pub'])
for year_x in papers_authors.year_pub.unique():
    authors = papers_authors[papers_authors.year_pub == year_x].author.values
    author_hind_at_year = author_h_index_in_year_X(authors, year_x)
    all_authors_hind = all_authors_hind.append(author_hind_at_year)
papers_authors = papers_authors.merge(all_authors_hind, how='left')

CPU times: user 15min 17s, sys: 2min 7s, total: 17min 24s
Wall time: 7min 14s


In [264]:
all_authors_hind.shape

(1799912, 3)

In [265]:
papers_authors.shape

(3308476, 3)

In [266]:
papers_authors['h-index'] = papers_authors['h-index'].fillna(0)

In [272]:
# for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
EARLY_CAREER = 3
combined_early_coauthor = combined[(combined.year_pub < combined.start_year + EARLY_CAREER)]

combined_early_coauthor = combined_early_coauthor.drop_duplicates(subset=['author', 'pub_id'])

combined_early_coauthor = combined_early_coauthor[['author', 'pub_id']]

# merging with self here removes coauthors that are not in their early career
combined_early_coauthor = combined_early_coauthor.merge(combined, on='pub_id')

combined_early_coauthor = combined_early_coauthor[combined_early_coauthor.author_x != combined_early_coauthor.author_y]
combined_early_coauthor = combined_early_coauthor.drop_duplicates(subset=['author_x', 'author_y'])

#here i merge the h-index_15. TODO: Replace this with h_index_in_year_of_publishing
# combined_early_coauthor = combined_early_coauthor.merge(credible_authors[['author', 'h-index_15']], left_on='author_y', right_on='author')

# combined_early_coauthor = combined_early_coauthor.groupby('author_x')['h-index_15'].max().reset_index()

# combined_early_coauthor.rename({"author_x":"author", "h-index_15": f"early_career_coauthor_max_hindex_{EARLY_CAREER}"}, 
#                              axis='columns', inplace=True)

# combined_early_coauthor = combined_early_coauthor[['author', f"early_career_coauthor_max_hindex_{EARLY_CAREER}"]]

# credible_authors = credible_authors.merge(combined_early_coauthor, on='author', how='left')
# credible_authors[f"early_career_coauthor_max_hindex_{EARLY_CAREER}"] = credible_authors[f"early_career_coauthor_max_hindex_{EARLY_CAREER}"].fillna(0)

In [275]:
combined_early_coauthor.shape

(4912811, 8)

In [277]:
combined_early_coauthor = combined_early_coauthor.merge(papers_authors, left_on=['author_y', 'year_pub'], 
                                                        right_on=['author', 'year_pub'])

In [279]:
combined_early_coauthor.head(20)

,author_x,pub_id,author_y,year_pub,id1,id2,year_cit,start_year,author,h-index
0,graeme earl,001c58d3-26ad-46b3-ab3a-c1e557d16821,gareth beale,2011,16870052-7e2e-4bbc-beeb-21e493bea221,001c58d3-26ad-46b3-ab3a-c1e557d16821,2015,2009,gareth beale,1
1,gareth beale,001c58d3-26ad-46b3-ab3a-c1e557d16821,graeme earl,2011,16870052-7e2e-4bbc-beeb-21e493bea221,001c58d3-26ad-46b3-ab3a-c1e557d16821,2015,2009,graeme earl,1
2,kathryn e. piquette,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
3,philip james basford,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
4,jacob dahl,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
5,alexander bischoff,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
6,michael hodgson,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
7,eleni kotoula,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
8,hembo pagi,6af505cf-727f-47f9-afea-772af2928086,graeme earl,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,2009,graeme earl,1
9,graeme earl,6af505cf-727f-47f9-afea-772af2928086,leif isaksen,2011,7be205bf-95e7-4baa-b800-770684720dec,6af505cf-727f-47f9-afea-772af2928086,2014,1973,leif isaksen,1


### Early Coauthor max citations

In [ ]:
for EARLY_CAREER in EARLY_CAREER_LEN_LIST:
    combined_early_coauthor = combined[(combined.year_pub < combined.start_year + EARLY_CAREER)]

    combined_early_coauthor = combined_early_coauthor.drop_duplicates(subset=['author', 'pub_id'])

    combined_early_coauthor = combined_early_coauthor[['author', 'pub_id']]

    combined_early_coauthor = combined_early_coauthor.merge(combined, on='pub_id')

    combined_early_coauthor = combined_early_coauthor[combined_early_coauthor.author_x != combined_early_coauthor.author_y]
    combined_early_coauthor = combined_early_coauthor.drop_duplicates(subset=['author_x', 'author_y'])

    combined_early_coauthor = combined_early_coauthor.merge(credible_authors[['author', 'succ_after_15y']], left_on='author_y', right_on='author')
    combined_early_coauthor = combined_early_coauthor.groupby('author_x')['succ_after_15y'].max().reset_index()

    combined_early_coauthor.rename({"author_x":"author", "succ_after_15y": f"early_career_coauthor_max_cit_{EARLY_CAREER}"}, 
                                 axis='columns', inplace=True)

    combined_early_coauthor = combined_early_coauthor[['author', f"early_career_coauthor_max_cit_{EARLY_CAREER}"]]

    credible_authors = credible_authors.merge(combined_early_coauthor, on='author', how='left')
    credible_authors[f"early_career_coauthor_max_cit_{EARLY_CAREER}"] = credible_authors[f"early_career_coauthor_max_cit_{EARLY_CAREER}"].fillna(0)

In [ ]:
combined_test = combined[(combined.year_pub < combined.start_year + 3)]

In [ ]:
credible_authors.sample(5)

In [ ]:
credible_authors.sample(1)['author'].values[0]

In [ ]:
# test - sample 50 authors and calculate data by hand, compare to sampled values to see if the calculation is good

In [ ]:
combined_test = combined[(combined.year_pub < combined.start_year + 3)]

In [ ]:
test_authors = credible_authors.sample(50)['author'].values

i = 1

for test_author in test_authors:

    test_author_data = credible_authors[credible_authors.author == test_author][
        ['author','early_career_degree_3', 'h-index_15', 'early_career_coauthor_max_hindex_3', 'early_career_coauthor_max_cit_3']]

    papers = combined_test[combined_test.author == test_author]['pub_id'].unique()

    authors = authorPublicationData[authorPublicationData.pub_id.isin(papers)].author
    
    coauth_data = credible_authors[credible_authors.author.isin(authors)][['author', 'h-index_15', 'succ_after_15y']]
    
    coauth_data = coauth_data[~coauth_data.author.isin(test_author_data['author'])]
    
    assert test_author_data['early_career_degree_3'].item() == coauth_data.shape[0], \
    f'Degree problem, expected {coauth_data.shape[0]}, got {test_author_data["early_career_degree_3"].item()}'
    
    assert test_author_data['early_career_coauthor_max_hindex_3'].item() == max(coauth_data["h-index_15"], default=0), \
    f'H index problem, expected {max(coauth_data["h-index_15"], default=0)}, got {test_author_data["early_career_coauthor_max_hindex_3"].item()}'
    
    assert test_author_data['early_career_coauthor_max_cit_3'].item() == max(coauth_data["succ_after_15y"], default=0), \
    f'15y success problem, expected {max(coauth_data["succ_after_15y"], default=0)}, got {test_author_data["early_career_coauthor_max_cit_3"].item()}'
    
    if i%10 == 0: 
        print(f"Passed test {i}")
    i += 1

In [ ]:
# drop
drop_list = ['early_career_recognition_EC3_RC3',
       'early_career_recognition_EC5_RC5']
credible_authors.drop(drop_list, axis=1, inplace=True)

In [ ]:
# How many authors does each paper have?
authorPublicationData.groupby('pub_id').agg({'author':'nunique'})

## Save

In [ ]:
# credible_authors.to_csv('derived-data/authors-scientific-atleast-'+str(CAREER_LENGTH)+'-year-extended.csv',
#                     index=False, encoding='utf-8')
credible_authors.to_csv('derived-data/authors-scientific-extended.csv',
                    index=False, encoding='utf-8')

In [ ]:
credible_authors.columns